In [1]:

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from collections import defaultdict
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch

import glob

from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing
import mkl_data_processing as mkldp
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
styleFiles = os.listdir(mpl.get_configdir())
styleFileIdx = 0

plt.style.use(os.path.join(mpl.get_configdir(), styleFiles[styleFileIdx]))
    

In [3]:
T7 = '/media/ak/T7/'
medians = os.path.join(T7, 'medians')

In [9]:
dfs = dict()
files = os.listdir(medians)
files

['ITV.L_medians.csv',
 'MKS.L_medians.csv',
 'KGF.L_medians.csv',
 'RBS.L_medians.csv']

In [11]:
for file in files:
    symbol = file.split("_")[0]
    dfs[symbol] = (pd.read_csv(os.path.join(medians, file)))

In [43]:
user_dict = dfs
columns_values = ['Hamming Loss', 'accuracy', 'f1- macro', 'f1- micro',
       'f1- weighted', 'precision', 'recall']
df =pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()},
                       orient='index')
## df.index.name = ("Symbol, Label")
## df.index.set_names(['Symbol', 'Label'], inplace=True)

In [44]:
df = df.rename(columns=lambda x : x + 1).stack().unstack(level=1)

In [45]:
df.index.set_names(['Symbol', 'Label'], inplace=True)

In [46]:
df

Hamming Loss  Unnamed: 0  accuracy  f1- macro  f1- micro  \
Symbol Label                                                             
ITV.L  1              0.19         1.0      0.81       0.48       0.81   
       2              0.38         2.0      0.62       0.45       0.62   
       3              0.64         3.0      0.36       0.30       0.36   
       4              0.31         4.0      0.69       0.47       0.69   
       5              0.66         5.0      0.34       0.31       0.34   
       6              0.58         6.0      0.42       0.29       0.42   
KGF.L  1              0.30         1.0      0.70       0.44       0.70   
       2              0.17         2.0      0.83       0.47       0.83   
       3              0.63         3.0      0.37       0.26       0.37   
       4              0.30         4.0      0.70       0.47       0.70   
       5              0.73         5.0      0.27       0.23       0.27   
       6              0.56         6.0      0.44       0.29       0.44   
MKS.L  1              0.61         1.0      0.39       0.30       0.39   
RBS.L  1              0.42         1.0      0.57       0.40       0.57   
       2              0.17         2.0      0.83       0.48       0.83   
       3              0.36         3.0      0.64       0.45       0.64   
       4              0.22         4.0      0.78       0.47       0.78   
       5              0.72         5.0      0.28       0.24       0.28   

              f1- weighted  precision  recall  
Symbol Label                                   
ITV.L  1              0.83       0.88    0.81  
       2              0.68       0.80    0.62  
       3              0.32       0.38    0.36  
       4              0.70       0.74    0.69  
       5              0.33       0.37    0.34  
       6              0.39       0.49    0.42  
KGF.L  1              0.79       0.96    0.70  
       2              0.81       0.81    0.83  
       3              0.29       0.36    0.37  
       4              0.71       0.77    0.70  
       5              0.27       0.50    0.27  
       6              0.40       0.47    0.44  
MKS.L  1              0.53       0.93    0.39  
RBS.L  1              0.68       0.94    0.57  
       2              0.80       0.81    0.83  
       3              0.69       0.77    0.64  
       4              0.75       0.75    0.78  
       5              0.27       0.45    0.28

In [47]:
print(df.to_latex())

\begin{tabular}{llrrrrrrrr}
\toprule
      &   &  Hamming Loss &  Unnamed: 0 &  accuracy &  f1- macro &  f1- micro &  f1- weighted &  precision &  recall \\
Symbol & Label &               &             &           &            &            &               &            &         \\
\midrule
ITV.L & 1 &          0.19 &         1.0 &      0.81 &       0.48 &       0.81 &          0.83 &       0.88 &    0.81 \\
      & 2 &          0.38 &         2.0 &      0.62 &       0.45 &       0.62 &          0.68 &       0.80 &    0.62 \\
      & 3 &          0.64 &         3.0 &      0.36 &       0.30 &       0.36 &          0.32 &       0.38 &    0.36 \\
      & 4 &          0.31 &         4.0 &      0.69 &       0.47 &       0.69 &          0.70 &       0.74 &    0.69 \\
      & 5 &          0.66 &         5.0 &      0.34 &       0.31 &       0.34 &          0.33 &       0.37 &    0.34 \\
      & 6 &          0.58 &         6.0 &      0.42 &       0.29 &       0.42 &          0.39 &       0.49 & 